<a href="https://colab.research.google.com/github/junghoum/Hello-world/blob/main/n222-random-forests/n222a-random-forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 2 / NOTE 2*

# 📝 Assignment
---

# 랜덤포레스트(Random Forests)

### 1) 캐글 대회를 이어서 진행합니다. EDA, 데이터 전처리 부분을 업데이트 하세요.
- EDA는 항상 완벽할 수 없지요 한 주간 계속 같은 데이터로 과제를 진행하므로 부족한 부분을 추가하거나 논하세요.
- (지금은 feature engineering에 너무 시간을 들이지 마세요!)
- Ordinal Encoding을 적용해 보세요.
- **(객관식) 다음 특성들 중에 순서를 고려한 Ordinal Encoding이 필요해 보이는 특성을 고르세요.**
    1. opinion_h1n1_vacc_effective
    2. state
    3. marital
    4. employment_occupation


In [92]:
import pandas as pd

target = 'vacc_h1n1_f'
# target = 'vacc_seas_f'
train = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'), 
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')
sample_submission = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/submission.csv')

In [93]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train[target], random_state=2)


train.shape, val.shape, test.shape

((33723, 39), (8431, 39), (28104, 38))

In [94]:
def engineer(df):
  behaviorals = [col for col in df.columns if 'behavioral' in col]
  df['behaviorals'] = df[behaviorals].sum(axis=1)

  dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]

  return df

train = engineer(train)
val = engineer(val)
test = engineer(test)

In [95]:
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [96]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()



In [97]:
train.head(10)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_insurance,health_worker,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,agegrp,education_comp,raceeth4_i,sex_i,inc_pov,marital,rent_own_r,employment_status,census_region,census_msa,n_adult_r,household_children,n_people_r,employment_industry,employment_occupation,hhs_region,state,vacc_h1n1_f,behaviorals
20365,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,Somewhat Effective,Somewhat Low,Not At All Worried,Very Effective,Somewhat Low,Not At All Worried,18 - 34 Years,4.0,3,1,1,2.0,1.0,Employed,1,"MSA, Not Principle City",4.0,0.0,4.0,"Professional, Scientific, and Technical Servic...",Office and Administrative Support Occupations,2,NEW JERSEY,0,0.0
29138,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,Somewhat Effective,Somewhat High,Not Very Worried,Somewhat Effective,Somewhat Low,Not Very Worried,35 - 44 Years,2.0,3,2,3,2.0,2.0,Unemployed,2,Non-MSA,2.0,2.0,4.0,NaN,NaN,7,IOWA,1,5.0
36128,2.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,Not Very Effective,Very Low,Dont Know,Somewhat Effective,Very Low,Very Worried,65+ Years,1.0,3,2,2,2.0,1.0,Not in Labor Force,4,"MSA, Not Principle City",1.0,0.0,1.0,NaN,NaN,10,OREGON,0,4.0
39793,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Very Effective,Somewhat Low,Not Very Worried,Somewhat Effective,Very Low,Not At All Worried,55 - 64 Years,4.0,3,1,1,1.0,1.0,Employed,2,"MSA, Principle City",2.0,0.0,2.0,"Arts, Entertainment, and Recreation Industries...",Business and Financial Operations Occupations,7,IOWA,0,3.0
32973,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Not At All Effective,Somewhat Low,Not At All Worried,Not At All Effective,Very Low,Not At All Worried,18 - 34 Years,4.0,3,2,2,1.0,1.0,Not in Labor Force,2,Non-MSA,2.0,3.0,5.0,NaN,NaN,7,IOWA,0,0.0
31450,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,Not Very Effective,Very Low,Not At All Worried,Not At All Effective,Very Low,Not At All Worried,18 - 34 Years,2.0,3,2,2,2.0,1.0,Employed,3,"MSA, Not Principle City",1.0,2.0,3.0,Not ascertained,Not ascertained,6,TEXAS,0,4.0
17541,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,Not At All Effective,Very Low,Somewhat Worried,Very Effective,Very High,Somewhat Worried,45 - 54 Years,3.0,3,2,3,1.0,2.0,Not in Labor Force,3,Non-MSA,3.0,0.0,3.0,NaN,NaN,4,GEORGIA,0,1.0
33946,2.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Dont Know,Somewhat Low,Not Very Worried,Somewhat Effective,Somewhat Low,Somewhat Worried,18 - 34 Years,4.0,3,2,4,1.0,1.0,Employed,1,"MSA, Principle City",1.0,1.0,2.0,Administrative and Support and Waste Management,Office and Administrative Support Occupations,1,CONNECTICUT,0,1.0
5300,2.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,Somewhat Effective,Somewhat Low,Not At All Worried,Somewhat Effective,Somewhat Low,Not At All Worried,55 - 64 Years,4.0,3,1,1,1.0,1.0,Not in Labor Force,4,"MSA, Not Principle City",2.0,0.0,2.0,NaN,NaN,10,OREGON,0,5.0
34292,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,0.0,Very Effective,Somewhat Low,Not At All Worried,Somewhat Effective,Very High,Somewhat Worried,55 - 64 Years,1.0,3,1,3,2.0,2.0,Employed,2,Non-MSA,1.0,0.0,1.0,Manufacturing Industries,Production Occupations,7,IOWA,0,1.0


In [98]:
!pip install --upgrade category_encoders



In [99]:
from category_encoders import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    OneHotEncoder(use_cat_names=True), 
    SimpleImputer(), 
    RandomForestClassifier(n_jobs=-1, random_state=10, oob_score=True)
)

pipe.fit(X_train, y_train)
print('검증 정확도: ', pipe.score(X_val, y_val))

검증 정확도:  0.8294389752105326


In [100]:
train_enc = train[['opinion_h1n1_vacc_effective']]

train_enc = train_enc.dropna(axis=0)

train_cat_encoded = enc.fit_transform(train_enc)
train_cat_encoded[:10]

array([[4.],
       [4.],
       [2.],
       [5.],
       [1.],
       [2.],
       [1.],
       [0.],
       [4.],
       [5.]])

In [101]:
train_enc.head(10)

,opinion_h1n1_vacc_effective
20365,Somewhat Effective
29138,Somewhat Effective
36128,Not Very Effective
39793,Very Effective
32973,Not At All Effective
31450,Not Very Effective
17541,Not At All Effective
33946,Dont Know
5300,Somewhat Effective
34292,Very Effective


### 2) 랜덤포레스트 모델을 적용한 후의 결과를 캐글에 제출하세요.
- 랜덤포레스트를 적용하고 성능이 오히려 떨어졌을 수도 있습니다! 만약 그렇다면 이유를 본인 논리로 분석해 보세요.
- **캐글 Leaderboard의 본인 Score를 제출하세요.**

In [102]:
import pandas as pd

from category_encoders import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    OneHotEncoder(use_cat_names=True), 
    SimpleImputer(), 
    RandomForestClassifier(n_jobs=-1, random_state=10, oob_score=True)
)

pipe.fit(X_train, y_train)
print('검증 정확도: ', pipe.score(X_val, y_val))

검증 정확도:  0.8294389752105326


In [103]:
pipe.named_steps['randomforestclassifier'].oob_score_

0.8219316193695697

In [106]:
forest = RandomForestClassifier(n_estimators=5, random_state=2)
forest.fit(X_train, y_train)

ValueError: ignored

In [113]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=100, noise=0.25, random_state=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,
 random_state=42)

forest = RandomForestClassifier(n_estimators=5, random_state=2)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5,
                       n_jobs=None, oob_score=False, random_state=2, verbose=0,
                       warm_start=False)

In [ ]:
print("훈련 세트 정확도: {:.3f}".format(forest.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(forest.score(X_test, y_test)))

In [ ]:
# from category_encoders import OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import make_pipeline
# from sklearn.metrics import f1_score

# pipe = make_pipeline(
#     OneHotEncoder(), 
#     SimpleImputer(), 
#     StandardScaler(), 
#     LogisticRegression(n_jobs=-1)
# )
# pipe.fit(X_train, y_train)

# print('검증세트 정확도', pipe.score(X_train, y_train))

# y_pred = pipe.predict(X_test)


In [ ]:
# pred = pipe.predict(X_train)
# print('F1 score: ', f1_score(y_train, pred))

In [114]:
pred_3 = pipe.predict(X_test)

df = pd.DataFrame({'ID':range(0,len(pred_3)),
                   'vacc_h1n1_f':pred_3})

df

ValueError: ignored

In [115]:
df.set_index('ID',inplace=True)
df

KeyError: ignored

In [116]:
df.to_csv('AI_222.csv')

## 🔥 도전과제

### 3) 수업에 사용하지 않은 다른 종류의 [category_encoders](http://contrib.scikit-learn.org/category_encoders/)을 적용해 보고 인코더에 대해 설명하세요


In [ ]:
### 이곳에서 과제를 진행해 주세요 ###

## 참고자료

- [Random Forests for Complete Beginners](https://victorzhou.com/blog/intro-to-random-forests/)